In [1]:
from __future__ import annotations

from math import ceil
from typing import TYPE_CHECKING, Literal

# Data retrieval
from anndata import AnnData
from lets_plot import (
    aes,
    geom_point,
    ggplot,
    ggtb,
    guide_legend,
    guides,
    labs,
    layer_tooltips,
)
from lets_plot.plot.core import PlotSpec

from cellestial.frames import build_frame
from cellestial.themes import _THEME_SCATTER
from cellestial.util import (
    _build_tooltips,
    _decide_tooltips,
    _determine_axis,
    _select_variable_keys,
)

if TYPE_CHECKING:
    from collections.abc import Iterable

    from lets_plot.plot.core import PlotSpec

In [2]:
def scatter(
    data: AnnData,
    x: str,
    y: str,
    *,
    axis: Literal[0,1] | None = None,
    color: str | None = None,
    fill: str | None = None,
    size: str | None = None,
    shape: str | None = None,
    point_color: str | None = None,
    point_fill: str | None = None,
    point_size: str | None = None,
    point_shape: str | None = None,
    interactive: bool = False,
    barcode_name: str = "Barcode",
    variable_name: str = "Varible",
    include_dimensions: bool = False,
    show_tooltips: bool = True,
    add_tooltips: Iterable[str] | str | None = None,
    custom_tooltips: Iterable[str] | str | None = None,
    tooltips_title: str | None = None,
    **point_kwargs,
) -> PlotSpec:
    """
    Scatter Plot.

    Parameters
    ----------
    data : AnnData
        The AnnData object of the single cell data.
    x : str
        The key for the x-axis.
    y : str
        The key for the y-axis.
    color : str | None, default=None
        Color aesthetic for the geom_point.
    fill : str | None, default=None
        Fill aesthetic for the geom_point.
    size : str | None, default=None
        Size aesthetic for the geom_point.
    shape : str | None, default=None
        Shape aesthetic for the geom_point.
    point_color : str | None, default=None
        Color for all the points.
        - Accepts:
            - hex code e.g. '#f1f1f1'
            - color name (of a limited set of colors).
            - RGB/RGBA e.g. 'rgb(0, 0, 255)', 'rgba(0, 0, 255, 0.5)'.
    point_fill : str | None, default=None
        Fill color for all the points.
        - Accepts:
            - hex code e.g. '#f1f1f1'
            - color name (of a limited set of colors).
            - RGB/RGBA e.g. 'rgb(0, 0, 255)', 'rgba(0, 0, 255, 0.5)'.
    point_size : str | None, default=None
        Size for all the points.
    point_shape : str | None, default=None
        Shape of all the points, an integer from 0 to 25.
        For more information see:
        https://lets-plot.org/python/pages/aesthetics.html#point-shapes
    interactive : bool, default=False
        Whether to make the plot interactive.
    barcode_name : str, default="Barcode"
        The name to give to barcode (or index) column in the dataframe.
    variable_name : str, default="Variable"
        The name to give to variable index column in the dataframe.
    include_dimensions : bool, default=False
        Whether to include dimensions in the dataframe.
    show_tooltips : bool, default=True
        Whether to show tooltips.
    add_tooltips : list[str] | tuple[str] | Iterable[str] | str | None, default=None
        Additional tooltips to show.
    custom_tooltips : list[str] | tuple[str] | Iterable[str] | str | None, default=None
        Custom tooltips, will overwrite the base_tooltips.
    tooltips_title : str | None, default=None
        Title for the tooltips.
    **point_kwargs : dict[str, Any]
        Additional parameters for the `geom_point` layer.
        For more information on geom_point parameters, see:
        https://lets-plot.org/python/pages/api/lets_plot.geom_point.html

    Returns
    -------
    PlotSpec
        Scatter plot.

    """
    # Handling Data types
    if not isinstance(data, AnnData):
        msg = "data must be an `AnnData` object"
        raise TypeError(msg)

    # handle point_kwargs
    if point_kwargs is None:
        point_kwargs = {}
    else:
        if "tooltips" in point_kwargs:
            msg = "use tooltips args within the function instead of adding `'tooltips' : 'value'` to `point_kwargs`\n"
            raise KeyError(msg)

    # Handle tooltips
    base_tooltips = [x,y]
    tooltips = _decide_tooltips(
        base_tooltips=base_tooltips,
        add_tooltips=add_tooltips,
        custom_tooltips=custom_tooltips,
        show_tooltips=show_tooltips,
    )

    # dimensional_keys = [key for key in [x, y] if key.startswith("X_")]
    keys = [key for key in [x, y, color, fill, size, shape] if key is not None and not key.startswith("X_")]
    variable_keys = _select_variable_keys(data=data, keys=keys)

    # BUILD: the dataframe
    axis = _determine_axis(data=data, keys=keys) if axis is None else axis
    frame = build_frame(
        data=data,
        variable_keys=variable_keys,
        axis=axis,
        observations_name=barcode_name,
        variables_name=variable_name,
        include_dimensions=include_dimensions,
    )

    # scatter kwargs
    point_kwargs["size"] = point_size
    point_kwargs["color"] = point_color
    point_kwargs["fill"] = point_fill
    point_kwargs["shape"] = point_shape

    # BUILD: the scatterplot
    scttr = (
        ggplot(data=frame)
        + geom_point(
            aes(x=x, y=y, color=color, size=size, shape=shape, fill=fill),
            tooltips=layer_tooltips(tooltips),
            **point_kwargs,
        )
        + labs(x=x, y=y)
        + _THEME_SCATTER
    )
    # handle legend wrapping
    if color is not None:
        n_distinct = frame.select(color).unique().height
        if n_distinct > 10:
            ncol = ceil(n_distinct / 10)
            scttr += guides(color=guide_legend(ncol=ncol))
    if fill is not None:
        n_distinct = frame.select(fill).unique().height
        if n_distinct > 10:
            ncol = ceil(n_distinct / 10)
            scttr += guides(fill=guide_legend(ncol=ncol))
    # handle interactive
    if interactive:
        scttr += ggtb()

    return scttr

In [3]:
import cellestial as cl
import scanpy as sc
from lets_plot import *

LetsPlot.setup_html()

data = sc.read("data/pbmc3k_pped.h5ad")

In [4]:
data.obs

,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,n_genes,leiden
AAACCCAAGGATGGCT-1,s1d1,2103,7.651596,8663.0,9.066932,42.721921,59.667552,69.744892,79.348955,460.0,6.133398,5.309938,3650.0,8.202756,42.133209,17.0,2.890372,0.196237,2103,0
AAACCCAAGGCCTAGA-1,s1d1,3916,8.273081,12853.0,9.461411,35.843772,44.262040,52.376877,62.763557,1790.0,7.490530,13.926710,1719.0,7.450079,13.374310,58.0,4.077538,0.451257,3916,10
AAACCCAAGTGAGTGC-1,s1d1,683,6.527958,1631.0,7.397562,56.284488,62.599632,70.386266,88.779890,581.0,6.366470,35.622318,63.0,4.158883,3.862661,13.0,2.639057,0.797057,683,15
AAACCCACAAGAGGCT-1,s1d1,4330,8.373554,17345.0,9.761117,27.662150,38.420294,48.901701,62.023638,780.0,6.660575,4.496973,3936.0,8.278174,22.692417,44.0,3.806663,0.253675,4330,17
AAACCCACATCGTGGC-1,s1d1,325,5.786897,555.0,6.320768,49.909910,59.459459,77.477477,100.000000,159.0,5.075174,28.648647,26.0,3.295837,4.684685,26.0,3.295837,4.684685,325,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGAGAGTCTGG-1,s1d3,277,5.627621,615.0,6.423247,63.089431,71.219512,87.479675,100.000000,282.0,5.645447,45.853661,35.0,3.583519,5.691057,20.0,3.044523,3.252032,277,6
TTTGTTGCAGACAATA-1,s1d3,3797,8.242230,13218.0,9.489410,30.753518,44.628537,53.691935,64.684521,864.0,6.762730,6.536541,3997.0,8.293550,30.239067,42.0,3.761200,0.317749,3797,16
TTTGTTGCATGTTACG-1,s1d3,3089,8.035926,27280.0,10.213945,64.409824,71.671554,78.317449,85.054985,1025.0,6.933423,3.757332,3562.0,8.178358,13.057184,13145.0,9.483872,48.185482,3089,5
TTTGTTGGTAGTCACT-1,s1d3,379,5.940171,698.0,6.549651,52.865330,60.028653,74.355301,100.000000,143.0,4.969813,20.487106,58.0,4.077538,8.309455,39.0,3.688879,5.587393,379,1


In [5]:

s = scatter(data,x="n_genes_by_counts",y="log1p_total_counts_hb",color="MT-ND2")
s

In [6]:
scatter(data,x="n_genes_by_counts",y="log1p_total_counts_hb",color="leiden")+scale_color_hue()

In [7]:
cl.retrieve(s)

Barcode,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,n_genes,leiden,MT-ND2
str,cat,i32,f64,f32,f32,f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64,cat,f32
"""AAACCCAAGGATGGCT-1""","""s1d1""",2103,7.651596,8663.0,9.066932,42.721921,59.667552,69.744892,79.348955,460.0,6.133398,5.309938,3650.0,8.202756,42.133209,17.0,2.890372,0.196237,2103,"""0""",0.031961
"""AAACCCAAGGCCTAGA-1""","""s1d1""",3916,8.273081,12853.0,9.461411,35.843772,44.26204,52.376877,62.763557,1790.0,7.49053,13.92671,1719.0,7.450079,13.37431,58.0,4.077538,0.451257,3916,"""10""",0.900468
"""AAACCCAAGTGAGTGC-1""","""s1d1""",683,6.527958,1631.0,7.397562,56.284488,62.599632,70.386266,88.77989,581.0,6.36647,35.622318,63.0,4.158883,3.862661,13.0,2.639057,0.797057,683,"""15""",1.27646
"""AAACCCACAAGAGGCT-1""","""s1d1""",4330,8.373554,17345.0,9.761117,27.66215,38.420294,48.901701,62.023638,780.0,6.660575,4.496973,3936.0,8.278174,22.692417,44.0,3.806663,0.253675,4330,"""17""",-0.324958
"""AAACCCACATCGTGGC-1""","""s1d1""",325,5.786897,555.0,6.320768,49.90991,59.459459,77.477477,100.0,159.0,5.075174,28.648647,26.0,3.295837,4.684685,26.0,3.295837,4.684685,325,"""6""",1.466883
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TTTGTTGAGAGTCTGG-1""","""s1d3""",277,5.627621,615.0,6.423247,63.089431,71.219512,87.479675,100.0,282.0,5.645447,45.853661,35.0,3.583519,5.691057,20.0,3.044523,3.252032,277,"""6""",1.454038
"""TTTGTTGCAGACAATA-1""","""s1d3""",3797,8.24223,13218.0,9.48941,30.753518,44.628537,53.691935,64.684521,864.0,6.76273,6.536541,3997.0,8.29355,30.239067,42.0,3.7612,0.317749,3797,"""16""",0.529579
"""TTTGTTGCATGTTACG-1""","""s1d3""",3089,8.035926,27280.0,10.213945,64.409824,71.671554,78.317449,85.054985,1025.0,6.933423,3.757332,3562.0,8.178358,13.057184,13145.0,9.483872,48.185482,3089,"""5""",-0.736241


In [8]:
data.var

,mt,ribo,hb,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,n_cells,highly_variable,means,dispersions,dispersions_norm,mean,std
AL390719.2,False,False,False,5,0.000292,0.000292,99.970803,5.0,1.791759,5,True,0.001001,1.840088,1.211725,0.000380,0.024219
C1QTNF12,False,False,False,62,0.003737,0.003730,99.637956,64.0,4.174387,62,True,0.008604,1.871498,1.274180,0.003594,0.069648
AL162741.1,False,False,False,26,0.001518,0.001517,99.848175,26.0,3.295837,26,True,0.003878,1.944082,1.418505,0.001503,0.046608
LINC01786,False,False,False,23,0.001343,0.001342,99.865693,23.0,3.178054,23,True,0.004039,1.990555,1.510911,0.001524,0.047831
AL391244.2,False,False,False,73,0.004438,0.004428,99.573723,76.0,4.343805,73,True,0.008619,1.879800,1.290687,0.003630,0.068932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PRKY,False,False,False,2294,0.165139,0.152840,86.604380,2828.0,7.947679,2289,True,0.346883,1.815441,1.162716,0.162510,0.463308
KDM5D,False,False,False,2030,0.140438,0.131412,88.145985,2405.0,7.785721,2028,True,0.283985,1.810605,1.153101,0.131121,0.414425
TTTY10,False,False,False,265,0.016876,0.016735,98.452555,289.0,5.669881,265,True,0.033677,1.812069,1.156013,0.014633,0.137948
MT-ND2,True,False,False,16324,23.342248,3.192213,4.677372,399736.0,12.898562,15951,True,3.988345,4.379118,1.680991,3.313847,1.351333


In [9]:
scatter(data,x="n_cells_by_counts",y="log1p_mean_counts",color="n_cells_by_counts")

In [10]:
s = scatter(data,x="n_genes_by_counts",y="log1p_total_counts_hb",color="X_UMAP1", include_dimensions=True,add_tooltips=["X_UMAP1"])
s